In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
customers = pd.read_csv("Customers.csv")
transactions = pd.read_csv("Transactions.csv")
products = pd.read_csv("Products.csv")

In [5]:
# Merge datasets to create a table
transactions_products = pd.merge(transactions, products, on="ProductID", how="left")
customer_data = pd.merge(transactions_products, customers, on="CustomerID", how="left")


In [8]:
# Create a pivot table: rows = customers, columns = products, values = total quantity
customer_product_matrix = customer_data.pivot_table(
    index="CustomerID", columns="ProductID", values="Quantity", fill_value=0
)

In [10]:
# Compute cosine similarity between customers
similarity_matrix = cosine_similarity(customer_product_matrix)

In [12]:
def recommend_customers(customer_id, customer_ids, similarity_matrix, top_n=3):
    try:       
        customer_idx = list(customer_ids).index(customer_id)
        similarity_scores = similarity_matrix[customer_idx]
        
        # Rank scores (excluding self)
        similar_customers = [
            (customer_ids[idx], similarity_scores[idx])
            for idx in np.argsort(similarity_scores)[::-1]
            if customer_ids[idx] != customer_id
        ]
        return similar_customers[:top_n]
    except ValueError:
        return []

In [18]:
# Generating lookalikes for customers C0001 to C0020
customer_ids = customer_product_matrix.index
lookalike_results = {}

for customer_id in customer_ids[:20]:  # First 20 customers
    recommendations = recommend_customers(customer_id, customer_ids, similarity_matrix)
    lookalike_results[customer_id] = recommendations

In [22]:
# Save results in the required format
lookalike_df = pd.DataFrame(
    {
        "CustomerID": lookalike_results.keys(),
        "Recommendations": [
            [(rec[0], round(rec[1], 3)) for rec in recs]
            for recs in lookalike_results.values()
        ],
    }
)

lookalike_df.to_csv("Bhuvanashree_H_J_Lookalike.csv", index=False)
print("Lookalike recommendations saved!")

Lookalike recommendations saved!
